# mpg

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime
import dateutil

In [ ]:
# Constants
win_size = 15                # size of the rolling window for calculating mpg
l_per_gal = 4.54             # litres in a gallon
from_csv = False
from_sqlite = not from_csv   # csv or sqlite, mutually exclusive
debug = False                # controls whether debug output is included
db_name = 'mpg.sqlite'
table_name = 'mpg'

In [ ]:
# read in mpg data from CSV
if from_csv:
    mpg_data = pd.read_csv('mpg.csv', header=0, index_col = 0, parse_dates=True, dayfirst=True, 
                          names=['Mileage', 'Fuel', 'Cost', 'Unit price'])

In [ ]:
# read in mpg data from sqlite db
if from_sqlite:
    engine = create_engine("sqlite:///"+db_name)
    mpg_data = pd.read_sql("SELECT * FROM "+table_name, engine)
    mpg_data.set_index('Date', inplace=True)
    mpg_data.index = pd.to_datetime(mpg_data.index, dayfirst=True)
    mpg_data.columns = ['Mileage', 'Fuel', 'Cost', 'Unit price']

In [ ]:
# peek at the data
if debug:
    print(mpg_data.head())
    print(type(mpg_data.index))
    print(mpg_data.index)

In [ ]:
# validate input
print('Input data\n----------')
print(mpg_data.head())

mpg_data['check_sum'] = (mpg_data['Fuel'] * mpg_data['Unit price']).round(decimals=2)
mpg_data['invalid'] = mpg_data['Cost'] != mpg_data['check_sum']
invalid_mpg = mpg_data[mpg_data['invalid']]
print('\nInvalid entries\n---------------')
print(invalid_mpg)

In [ ]:
mpg_data['mpg'] = (l_per_gal * mpg_data['Mileage'].diff(periods=win_size-1)/mpg_data['Fuel'].rolling(window=win_size).sum()).round(decimals=2)
if debug:
    print(mpg_data.head(2*win_size))
    print(mpg_data.tail(win_size))

In [ ]:
# plot mpg
sns.set()
mpg_data['mpg'][win_size-1:].plot(figsize=(12,8))
_ = plt.xlabel('date')
_ = plt.ylabel('mpg')
_ = plt.ylim((30,65))
#_ = plt.axis('tight')
# TODO more uniform sampling - does this go back to (upsampling) resampling and interpolation of eariler data
dates = mpg_data.index.to_numpy()[::win_size]
if debug:
    print(dates)
labels = np.datetime_as_string(dates, unit='M')
_ = plt.xticks(dates, labels, rotation=60)
_ = plt.title('mpg')
print(mpg_data.tail())
#plt.grid(True)

Annual mileage

In [ ]:
annual_mileage = pd.DataFrame(mpg_data['Mileage'].resample('D').interpolate('time'))
annual_mileage['Annual mileage'] = annual_mileage.diff(periods=365)
annual_mileage = annual_mileage.resample('M').min()
annual_mileage = annual_mileage.drop('Mileage', axis=1)
annual_mileage = annual_mileage[12:]                      # remove NaN rows
if debug:
    print(annual_mileage.head())
sns.set()
annual_mileage.plot(figsize=(12,8), legend=False)
_ = plt.xlabel('date')
_ = plt.ylabel('mileage')
_ = plt.title('Annual mileage')

Monthly fuel spend (moving average)

In [ ]:
monthly_spend = pd.DataFrame(mpg_data['Cost'].resample('M').sum()).rolling(6, center=True).mean()
if debug:
    print(monthly_spend.head())
    print(monthly_spend.tail())
sns.set()
monthly_spend.plot(figsize=(18,6), legend=False)
_ = plt.xlabel('date')
_ = plt.ylabel('fuel spend')
_ = plt.title('Monthly fuel spend (moving average)')